# Feature Reduction

### Todo

Add PCA and AutoEncoder funtionality.

How do we deal with outliers

Derived statistics: Sloope, Average, Max/Min

Discuss HVLT correlation/should we use them with andrijana

Automatic naming of folder with timestamp

explore standardization before and after longitudinal statistics

#### This cell is for defining various OPTIONS used for this notebook (working directory, how many rows and columns pandas displays for a dataframe, etc). 

#### Preferably this cell is also where we do important imports (for example pandas and numpy)

In [162]:
import os 
#Input the directory where your joined_data.csv is located 
#os.chdir('C:/Users/Trond/Documents/Master 2020/Processed data')
os.chdir('C:/Users/Briggstone/Documents/Master 2020/Processed data')
# os.chdir('C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data')

#Where you want the csv file of the merged data to be placed
output_filepath = 'C:/Users/Briggstone/Documents/Master 2020/Processed data'
#output_filepath = 'C:/Users/Trond/Documents/Master 2020/Processed data'
#output_filepath = 'C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data'

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.model_selection import train_test_split

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 1000)




'''
Various flags for which preprocessing steps to take and what graphics we want to show:
'''

'''
    Flags for which missing value imputation to use
'''
    
# 0 = MODE/MEAN IMPUTATION, 1 = SIMILARITY MEASURE
MV_FLAG = 0    

'''
'''
    

'''
    Flags for outlier detection and removal
'''
# The ratio of outliers that we expect in our data and consquently will be classified as such by Isolation Forest
OUTLIER_RATIO = 0.01
    
# 0 = No, 1 = Yes
SHOW_OUTLIER_STATISTICS = 1

# 0 = No, 1 = Yes
SHOW_OUTLIER_OBSERVATIONS = 0

# 0 = No, 1 = Yes
REMOVE_OUTLIERS = 0

'''
'''


'''
    Flags for graphical representations of distributions and pearson correlation heatmaps
'''

# 0 = No, 1 = Yes
SHOW_DISTRIBUTIONS = 0

# 0 = No, 1 = Yes
SHOW_HEATMAP = 0

'''
'''

'''
    Flags for how we split our data into training and test sets
'''

#The portion of data in the test set
TEST_PORTION = 0.2

'''
'''

'''
    Flags for how we standardize and one-hot encode our variables
'''

#Whether to leave one out in one-hot encoding, e.g dropping MALE in MALE/FEMALE because MALE can be infered from FEMALE = 0
# 0 = No, 1 = Yes
ONE_HOT_LEAVE_ONE_OUT = 0

#Whether we standarize ordinal variables or not
# 0 = No, 1 = Yes
STANDARDIZE_ORDINAL = 0

'''
    Flags for how we process our components variables
'''
# 0 = no derived total scores, 1 = derived total scores with no dichotomization, 2 = derived total scores with dichotomization
DERIVED_TOTAL_SCORES = 2
'''
'''

'''
    Flags for dimensionality reduction algorithms
'''

# 0 = no dimensionality reduction, 1 = PCA 
DIMENSIONALITY_REDUCTION = 0

if DIMENSIONALITY_REDUCTION == 1:
    #Set ratio of variance to keep for PCA dimensions, translates to the number of dimensions of PCA kept
    VARIANCE_KEEP = 0.9
    
'''
'''

'''
    Flags for how we deal with longitudinal data
'''
    
# 0 = Only Baseline data, 1 = i-approach

LONGITUDINAL_STATISTICS = 1

'''
degree [0,x], -1 if you don't want intercept
average, 0 = No, 1= Yes
max, 0 = No, 1= Yes
min, 0 = No, 1= Yes
'''
if LONGITUDINAL_STATISTICS != 0:
    NUMERIC_APPROACH = {
        "degree": 1,
        "average": 1,
        "max" : 1,
        "min" : 1
    }
'''
'''

#Just to stop \n being printed out due to the open strings
clear = 1

#### In this cell we import our joined imputed data based on MV_FLAG

In [163]:
if MV_FLAG == 0:
    data = pd.read_csv('joined_data_mm.csv') # missing values filled with mean/median
else: 
    data = pd.read_csv('joined_data_heom.csv') # missing values filled based on HEOM measure



#### In this cell we define a function for outlier detection and removal using Isolation Forest
#### We call this function with the flags set at start

In [164]:
from sklearn.ensemble import IsolationForest

#ratio_to_remove is the important variable that determines the ratio of observations to remove as outliers
def outliers_IF (df, outlier_ratio, verbose, remove, show_outliers):
    tempdata = df.drop(["PATNO", "LAST_EVENT_ID", "EVENT_ID"], axis = 1)
    df["OUTLIERS_PRED"] = IsolationForest(random_state = 0, contamination = outlier_ratio).fit_predict(tempdata)
    if verbose:
        print("Total datapoints: ", df.shape[0])
        print("Datapoints flagged as outliers: ", df.loc[df.OUTLIERS_PRED == -1, :].shape[0])
        print("Percentage of datapoints flagged as outliers: ", df.loc[df.OUTLIERS_PRED == -1, :].shape[0] / df.shape[0])

        print("\nDatapoints flagges as outliers who are of positive class: ", df.loc[(df.OUTLIERS_PRED == -1) & (df.NP1HALL != 0), :].shape[0])
        print("Percentage of positive class who are classified as outliers:", df.loc[(df.OUTLIERS_PRED == -1) & (df.NP1HALL != 0), :].shape[0] / df.loc[df.NP1HALL != 0, :].shape[0])

        print("\nDatapoints flagges as outliers who are of negative class: ", df.loc[(df.OUTLIERS_PRED == -1) & (df.NP1HALL == 0), :].shape[0])
        print("Percentage of negative class who are classified as outliers:", df.loc[(df.OUTLIERS_PRED == -1) & (df.NP1HALL == 0), :].shape[0] / df.loc[df.NP1HALL == 0, :].shape[0])
     
    if show_outliers:
        print("Observations which are classified as outliers with current outlier ratio of ", str(outlier_ratio))
        display(df.loc[df.OUTLIERS_PRED == -1, :])
    
    if remove:
        df.drop(df[df.OUTLIERS_PRED == -1].index, inplace = True)
        
        
        
    df.drop("OUTLIERS_PRED", axis = 1, inplace = True)
    

outliers_IF (data,OUTLIER_RATIO, SHOW_OUTLIER_STATISTICS, REMOVE_OUTLIERS, SHOW_OUTLIER_OBSERVATIONS)


Total datapoints:  2258
Datapoints flagged as outliers:  23
Percentage of datapoints flagged as outliers:  0.010186005314437556

Datapoints flagges as outliers who are of positive class:  9
Percentage of positive class who are classified as outliers: 0.052941176470588235

Datapoints flagges as outliers who are of negative class:  14
Percentage of negative class who are classified as outliers: 0.006704980842911878


#### In this cell we define a function that shows us interesting distributions for our response

In [165]:
def show_dist (data):
    data_HALL_AFTER = data.copy(deep = True)
    unique_patno = data.PATNO.unique()
    event_ids = np.sort(data.EVENT_ID.unique())
    for x in unique_patno:
        subject_data = data_HALL_AFTER.loc[data_HALL_AFTER.PATNO == x, ["EVENT_ID", "HALL"]]
        subject_event_ids = np.sort(subject_data.EVENT_ID.unique())
        first_hall = 0
        for i in subject_event_ids:
            if first_hall != 1 and subject_data.loc[subject_data.EVENT_ID == i, ["HALL"]].values[0] == 1:
                    first_hall = 1
            elif first_hall == 1:
                    data_HALL_AFTER.loc[(data_HALL_AFTER.PATNO == x) & (data_HALL_AFTER.EVENT_ID == i), ["HALL"]] = 1

    event_id_to_years = {
        "BL" : "Baseline",
        "V04": "1-Year",
        "V06": "2-Year",
        "V08": "3-Year",
        "V10": "4-Year",
        "V12": "5-Year",
        "V13": "6-Year",
        "V14": "7-Year",
        "V15": "8-Year"   
    }

    '''
    distribution1 = Total subjects in the study per year
    distribution2 = Subjects with a non-missing datapoint per year
    distribution3 = Percentage of remaining subjects in the study with a non-missing datapoint per year
    distribution4 = Number of subjects with a non-missing datapoint that report hallucinations per year
    distribution5 = Number of subjects with a non-missing datapoint that report hallucinations or have reported hallucinations a previous years per year
    distribution6 = Percentage of subjects with a non-missing datapoint that report hallucinations per year
    distribution7 = Percentage of subjects with a non-missing datapoint that report hallucinations or have reported hallucinations previous years per year
    distribution8 = Number of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    distribution9 = Percentage of previously non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    '''

    #Total subjects in the study per year
    distribution1 = {}

    subjects = data.PATNO.unique().size
    for x in event_ids:
        distribution1 [event_id_to_years[x]] = subjects
        subjects -= data.loc[data.LAST_EVENT_ID == x, "PATNO"].unique().size


    #Subjects with a datapoint per year
    distribution2 = {}

    for x in event_ids:
        distribution2 [event_id_to_years[x]] = data.loc[data.EVENT_ID == x, "PATNO"].unique().size



    #Percentage of remaining subjects with a datapoint per year       
    distribution3 = {}
    subjects = data.PATNO.unique().size
    for x in event_ids:
        distribution3 [event_id_to_years[x]] = (data.loc[data.EVENT_ID == x, "PATNO"].unique().size / subjects) * 100
        subjects -= data.loc[data.LAST_EVENT_ID == x, "PATNO"].unique().size


    # Number of subjects with a datapoint that report hallucinations per year
    distribution4 = {}
    for x in event_ids:
        subjects_hall = data.loc[(data.EVENT_ID == x) & (data.HALL == 1), "PATNO"].values
        distribution4[event_id_to_years[x]] = subjects_hall.size 


    #Number of subjects with a datapoint that report hallucinations or have reported hallucinations priorly per year
    distribution5 = {}

    for x in event_ids:
        subjects_hall = data_HALL_AFTER.loc[(data_HALL_AFTER.EVENT_ID == x) & (data_HALL_AFTER.HALL == 1), "PATNO"].values
        distribution5[event_id_to_years[x]] = subjects_hall.size 


    #Percentage of subjects with a non-missing datapoint that report hallucinations per year
    distribution6 = {}

    for x in event_ids:
        subjects_hall = data.loc[(data.EVENT_ID == x) & (data.HALL == 1), "PATNO"].values
        distribution6[event_id_to_years[x]] = (subjects_hall.size / data.loc[data.EVENT_ID == x, "PATNO"].unique().size) * 100

    #Percentage of subjects with a non-missing datapoint that report hallucinations or have reported hallucinations previous years per year
    distribution7 = {}

    for x in event_ids:
        subjects_hall = data_HALL_AFTER.loc[(data_HALL_AFTER.EVENT_ID == x) & (data_HALL_AFTER.HALL == 1), "PATNO"].values
        distribution7[event_id_to_years[x]] = (subjects_hall.size / data.loc[data.EVENT_ID == x, "PATNO"].unique().size) * 100

    #Number of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    distribution8 = {}

    tempdata = data_HALL_AFTER.copy(deep = True)
    for x in event_ids: 
        subjects_hall = tempdata.loc[(tempdata.EVENT_ID == x) & (tempdata.HALL == 1), "PATNO"].values
        distribution8[event_id_to_years[x]] = subjects_hall.size

        for i in subjects_hall:
            tempdata = tempdata[tempdata.PATNO != i]

    #Percentage of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    distribution9 = {}

    tempdata = data_HALL_AFTER.copy(deep = True)
    for x in event_ids: 
        subjects_hall = tempdata.loc[(tempdata.EVENT_ID == x) & (tempdata.HALL == 1), "PATNO"].values
        distribution9[event_id_to_years[x]] = (subjects_hall.size / data.loc[data.EVENT_ID == x, "PATNO"].unique().size) * 100

        for i in subjects_hall:
            tempdata = tempdata[tempdata.PATNO != i]






    fig, axs = plt.subplots(5, 2)
    fig.set_size_inches(25, 25)
    fig.tight_layout(pad=5.0)
    fig.delaxes(axs[4,1])


    # distribution1 = Total subjects in the study per year
    axs[0,0].bar(distribution1.keys(), distribution1.values(), width=.5, color='g')
    axs[0,0].title.set_text("Total subjects in the study per year")

    #distribution2 = Subjects with a non-missing datapoint per year
    axs[0,1].bar(distribution2.keys(), distribution2.values(), width=.5, color='r')
    axs[0,1].title.set_text("Subjects with a non-missing datapoint per year")

    #distribution3 = Percentage of remaining subjects in the study with a non-missing datapoint per year
    axs[1,0].bar(distribution3.keys(), distribution3.values(), width=.5, color='y')
    axs[1,0].title.set_text("Percentage of remaining subjects in the study with a non-missing datapoint per year")

    #distribution4 = Number of subjects with a non-missing datapoint that report hallucinations per year
    axs[1,1].set_ylim(0,70)
    axs[1,1].bar(distribution4.keys(), distribution4.values(), width=.5, color='k')
    axs[1,1].title.set_text("Number of subjects with a non-missing datapoint that report hallucinations per year")

    #distribution5 = Number of subjects with non-missing datapoint that report hallucinations \n or have reported hallucinations a previous year per year
    axs[2,0].set_ylim(0,70)
    axs[2,0].bar(distribution5.keys(), distribution5.values(), width=.5, color='c')
    axs[2,0].title.set_text("Number of subjects with non-missing datapoint that report hallucinations \n or have reported hallucinations a previous year per year")

    #distribution6 = Percentage of subjects with a non-missing datapoint that report hallucinations per year
    axs[2,1].set_ylim(0,50)
    axs[2,1].bar(distribution6.keys(), distribution6.values(), width=.5, color='m')
    axs[2,1].title.set_text("Percentage of subjects with a non-missing datapoint that report hallucinations per year")

    #distribution7 = Percentage of subjects with a non-missing datapoint that report hallucinations or have reported hallucinations a previous year per year
    axs[3,0].set_ylim(0,50)
    axs[3,0].bar(distribution7.keys(), distribution7.values(), width=.5, color= '#3933FF')
    axs[3,0].title.set_text("Percentage of subjects with a non-missing datapoint that report hallucinations \n or have reported hallucinations a previous year per year")

    #Number of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    axs[3,1].set_ylim(0,30)
    axs[3,1].bar(distribution8.keys(), distribution8.values(), width=.5, color= '#EE910C')
    axs[3,1].title.set_text("Number of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year")

    #Percentage of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    axs[4,0].set_ylim(0,20)
    axs[4,0].bar(distribution9.keys(), distribution9.values(), width=.5, color= '#978A8A')
    axs[4,0].title.set_text("Percentage of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year")


#### In this cell we dichtomize our response and look at various distributions if flag is set

In [166]:
'''Dihotomize NP1HALL dependent variable'''

# if the patient has not suffered hallucinations, we consider it 0
# if the patient has suffered >= 1 times hallucinations, we consider it 1   
data['HALL'] = data['NP1HALL'].apply(lambda x: np.where(x >=1, 1, 0))

data.drop('NP1HALL', inplace = True, axis = 1)
 
if SHOW_DISTRIBUTIONS == 1:
    show_dist(data)


#### In this cell we define the functions we need to calculate derived values from various variables

In [167]:
#Functions for calculating derived values for various data tables

def lnsqtot(df):
    ''' 
    Letter number sequencing 
    sum variables
    '''
    component_vars = ["LNS1A", "LNS1B", "LNS1C", \
        "LNS2A", "LNS2B", "LNS2C", "LNS3A", "LNS3B", "LNS3C", "LNS4A", "LNS4B", "LNS4C", "LNS5A", "LNS5B", "LNS5C", "LNS6A", "LNS6B", "LNS6C", \
        "LNS7A", "LNS7B", "LNS7C"]
                      
    df["LNSTOT"] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)                     
    
    

def hvlttot (df):
    
    component_vars = ["HVLTRT1", "HVLTRT2", "HVLTRT3", "HVLTREC", "HVLTFPRL", "HVLTFPUN"]
    
    #Immediate/Total recall
    df["HVLT_TOTAL"] = df.loc[:,["HVLTRT1", "HVLTRT2", "HVLTRT3"]].sum(axis = 1)
    
    #Discrimination Recognition
    df["HVLT_DISCRIM"] = df.HVLTREC - (df.HVLTFPRL + df.HVLTFPUN)
    
    #Retention
    df["HVLT_RETENTION"] = df.HVLTRDLY / df.loc[:,["HVLTRT2", "HVLTRT3"]].max(axis = 1)
    
    df.drop(component_vars,inplace = True, axis = 1)
    
    
def mcatot (df):
    ''' 
    Montreal cognitive test
    sum variables and 1 point to score if education years <= 12 and score < 30
    '''
    
    component_vars = ["MCAALTTM", "MCACUBE", "MCACLCKC", "MCACLCKN", "MCACLCKH", "MCALION", "MCARHINO", "MCACAMEL", \
    "MCAFDS", "MCABDS", "MCAVIGIL", "MCASER7", "MCASNTNC", "MCAVF", "MCAABSTR", "MCAREC1", "MCAREC2", "MCAREC3", "MCAREC4", \
    "MCAREC5", "MCADATE", "MCAMONTH", "MCAYR", "MCADAY", "MCAPLACE", "MCACITY"]
    
    df['MCATOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    
    df['MCATOT'] = df.apply(lambda row_wise: row_wise["MCATOT"] + 1 if row_wise["EDUCYRS"] <= 12 and row_wise["MCATOT"] < 30 else row_wise["MCATOT"] , axis=1)   
    

def vlttot (df):
    ''' Semantic Fluency
    VLTANIM, VLTVEG,VLTFRUIT need to be summed in order to obtain a final score'''
    
    component_vars = ["VLTANIM", "VLTVEG", "VLTFRUIT"]
    
    df['VLTTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

    
def remqtot (df):
    '''REM sleep behavior disorder (RBD)'''
    
    component_vars = ["STROKE","HETRA", "PARKISM", "RLS", "NARCLPSY", "DEPRS", "EPILEPSY", "BRNINFM", "CNSOTH"]
        
    score = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    
    # 1 point if any of these component variables had a 1, else 0
    score = pd.Series(np.where(score >= 1, 1, 0))
    
    df.drop(component_vars, inplace = True, axis = 1)

    component_vars = ["DRMVIVID", "DRMAGRAC", "DRMNOCTB", "SLPLMBMV", "SLPINJUR", \
                      "DRMVERBL", "DRMFIGHT", "DRMUMV", "DRMOBJFL", "MVAWAKEN", "DRMREMEM", "SLPDSTRB"]
    
    score += df.loc[:, component_vars].sum(axis = 1, skipna = False)

    df['REMTOT'] = score
    df.drop(component_vars, inplace = True, axis = 1)

    
def gdsstot (df):
    '''Geriatric Depression Scale'''
    
    component_vars = ["GDSSATIS", "GDSDROPD", \
    "GDSEMPTY", "GDSBORED", "GDSGSPIR", "GDSAFRAD", "GDSHAPPY", "GDSHLPLS", "GDSHOME", "GDSMEMRY", "GDSALIVE", "GDSWRTLS", "GDSENRGY", \
    "GDSHOPLS", "GDSBETER"]
    
    df['GDSSTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

    
def sidttot (df):
    '''Olfactory impairment: University of Pennsylvania Smell ID Test'''
    component_vars = ["UPSITBK1", "UPSITBK2", "UPSITBK3", "UPSITBK4"]
    
    df['SIDTTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

    
def epsstot (df):
    '''Epworth Sleepiness Scale'''
    
    component_vars = ["ESS1", "ESS2", \
    "ESS3", "ESS4", "ESS5", "ESS6", "ESS7", "ESS8"]
    
    df['EPSSTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

    
def scoptot(df):
    '''Scales for Outcomes in Parkinson’s Disease–Autonomic'''

    component_vars = ["SCAU1", "SCAU2", \
    "SCAU3", "SCAU4", "SCAU5", "SCAU6", "SCAU7", "SCAU8", "SCAU9", "SCAU10", "SCAU11", "SCAU12", "SCAU13", \
    "SCAU14", "SCAU15", "SCAU16", "SCAU17", "SCAU18", "SCAU19", "SCAU20", "SCAU21", "SCAU22", "SCAU23", "SCAU24", "SCAU25"]
    
    df['SCOPTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    

def msu3tot(df):
    '''Movement Disorders Society–Unified Parkinson Disease Rating Scale'''
    
    component_vars = ['NP3BRADY', 'NP3FACXP', 'NP3FRZGT', \
    'NP3FTAPL', 'NP3FTAPR', 'NP3GAIT', 'NP3HMOVL', 'NP3HMOVR', 'NP3KTRML', 'NP3KTRMR', 'NP3LGAGL', 'NP3POSTR', 'NP3PRSPL', 'NP3PRSPR', \
    'NP3PSTBL', 'NP3PTRML', 'NP3PTRMR', 'NP3RIGLL', 'PN3RIGRL', 'NP3RIGN', 'NP3RIGRU', 'NP3RISNG', 'NP3RTALJ', 'NP3RTALL', 'NP3RTALU', \
    'NP3RTARL', 'NP3RTARU', 'NP3RTCON', 'NP3SPCH', 'NP3TTAPL', 'NP3TTAPR', 'NP3LGAGR', 'NP3RIGLU']
       
    df['MSU3TOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    #df.drop(component_vars, inplace = True, axis = 1) #cannot drop, variables needed in tremor and pigd

    
def tremor(df):
    '''Tremor score'''
    
    component_vars = ["NP2TRMR", "NP3PTRMR", "NP3PTRML", "NP3KTRMR", "NP3KTRML", "NP3RTARU", "NP3RTALU", "NP3RTARL", "NP3RTALL", \
    "NP3RTALJ", "NP3RTCON"]
    
    df['TREMOR'] = df.loc[:, component_vars].mean(axis = 1, skipna = False)
    #df.drop(component_vars, inplace = True, axis = 1) #cannot drop, variables needed in tremor and pigd
    
    
def pigd(df):
    '''PIGD score'''
    
    component_vars = ["NP2WALK", "NP2FREZ", "NP3GAIT", "NP3FRZGT", "NP3PSTBL"]
    df['PIGD'] = df.loc[:, component_vars].mean(axis = 1, skipna = False)
    
    component_vars = ['NP3BRADY', 'NP3FACXP', 'NP3FRZGT', \
    'NP3FTAPL', 'NP3FTAPR', 'NP3GAIT', 'NP3HMOVL', 'NP3HMOVR', 'NP3KTRML', 'NP3KTRMR', 'NP3LGAGL', 'NP3POSTR', 'NP3PRSPL', 'NP3PRSPR', \
    'NP3PSTBL', 'NP3PTRML', 'NP3PTRMR', 'NP3RIGLL', 'PN3RIGRL', 'NP3RIGN', 'NP3RIGRU', 'NP3RISNG', 'NP3RTALJ', 'NP3RTALL', 'NP3RTALU', \
    'NP3RTARL', 'NP3RTARU', 'NP3RTCON', 'NP3SPCH', 'NP3TTAPL', 'NP3TTAPR', "NP2TRMR", "NP2WALK", "NP2FREZ", 'NP3LGAGR', 'NP3RIGLU']
    #cannot drop before we discuss missing values
    df.drop(component_vars, inplace = True, axis = 1) #drop everything from msu3tot, tremor and pig
    
def famtot (df):
    ''' Raw sum of PD family history'''
    component_vars = ["BIOMOMPD", "BIODADPD", "FULSIBPD", "HAFSIBPD", "MAGPARPD", "PAGPARPD", "MATAUPD", "PATAUPD", "KIDSPD"]   
    df["FAMTOT"] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

#### In this cell we define functions for encoding and dichotomizing variables

In [168]:
#Functions for encoding and dichotomizing variables

def ratio(x, y):
    ''' Calculate TD/PGID ratio'''
    
    if y == 0:
        if x == 0:
            ratio = 0 #indeterminate
        else: 
            ratio =1 #TD
    elif x/y >= 1.15:
        ratio = 1 #TD
    elif x/y <= 0.9:
        ratio = 2 #PIGD
    else:
        ratio = 0 #indeterminate 
    return ratio

def td_pigd_classification(df):
    '''Tremor/PIGD ratio'''
    
    component_vars = ['TREMOR', 'PIGD']
    df['TD_PIGD_RATIO'] = df.apply(lambda x: ratio(x['TREMOR'], x['PIGD']), axis=1)
    df.drop(component_vars, inplace = True, axis = 1)
   

def famhist(df):
    '''Family history of Parkinson's Disease'''
    
        
    score = df.FAMTOT
    
    # if score >= 1 then 1, else 0
    # if score = NaN, then 0
    score = pd.Series(np.where(score >= 1, 1, 0))
    
    df.drop(["FAMTOT"], inplace = True, axis = 1)
    df['FAMHIST'] = score
    
    
def sleepy(df):
    '''Dichotomize EPSSTOT, Epworth Sleepiness Scale'''
    
    # if score < 10 subjects will be classified as 0 (not sleepy)
    # if score >= 10 subject will be classified as 1 (sleepy).
    df['SLEEPY'] = df['EPSSTOT'].apply(lambda x: np.where(x >=10, 1, 0))

    df.drop('EPSSTOT', inplace = True, axis = 1)


def depr(df):
    '''Dichotomize GDSSTOT, Geriatric Depression Scale'''
    
    # if score <5 subjects will be classified as 0 (non-depressed).
    # if score >= 5 subjects will be classified as 1 (depressed) 
    df['DEPR'] = df['GDSSTOT'].apply(lambda x: np.where(x >=5, 1, 0))

    df.drop('GDSSTOT', inplace = True, axis = 1)


def rbd(df):
    '''Dichotomize REMTOT, REM sleep behavior disorder (RBD)'''
       
    # if score <5 subjects will be classified as 0 (RBD negative).
    # if score >= 5 subjects will be classified as 1 (RBD positive) 
    df['RBD'] = df['REMTOT'].apply(lambda x: np.where(x >=5, 1, 0))

    df.drop('REMTOT', inplace = True, axis = 1)

#### Derived_values and dichotomization wrapper function

In [169]:
def derived_values(df, dichotomize):
    '''All derived_values preprocessing together'''
    lnsqtot(df)
    hvlttot(df)
    mcatot(df)
    vlttot(df)
    remqtot(df)
    gdsstot(df)
    sidttot(df)
    epsstot(df)
    scoptot(df)
    msu3tot(df)
    tremor(df)
    pigd(df)
    famtot(df)
    
    if dichotomize == True:
        famhist(df)
        sleepy(df)
        depr(df)
        rbd(df)  
        td_pigd_classification(df)

#### In this cell we define our function for standardizing numerical and ordinal variables and one-hot encoding categorical ones

In [170]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

def PD_MED_USE_ONEHOT (PD_SERIES, leave_one_out):
    
    df = pd.DataFrame()
    df["PD_MED_USE"] = PD_SERIES
    if not leave_one_out:
        df["C_MED_NO"] = 0
    df["C_MED_LEV"] = 0
    df["C_MED_AG"] = 0
    df["C_MED_OTHER"] = 0
    
    for index, row in df.iterrows():
        value = row["PD_MED_USE"]
        if value == 0 and not leave_one_out:
            df.at[index,"C_MED_NO"] = 1
        if value == 1:
            df.at[index,"C_MED_LEV"] = 1
        elif value == 2:
            df.at[index,"C_MED_AG"] = 1
        elif value == 3:
            df.at[index,"C_MED_OTHER"] = 1
        elif value == 4: 
            df.at[index,"C_MED_LEV"] = 1
            df.at[index,"C_MED_OTHER"] = 1
        elif value == 5:
            df.at[index,"C_MED_LEV"] = 1
            df.at[index,"C_MED_AG"] = 1    
        elif value == 6:
            df.at[index,"C_MED_AG"] = 1
            df.at[index,"C_MED_OTHER"] = 1  
        elif value == 7:
            df.at[index,"C_MED_LEV"] = 1
            df.at[index,"C_MED_AG"] = 1   
            df.at[index,"C_MED_OTHER"] = 1 
    df.drop("PD_MED_USE", inplace = True, axis = 1)
    return df

    
def standardize (numeric_train, numeric_test):
    
    scaler = StandardScaler()
    scaler.fit(numeric_train)
    
    numeric_train[numeric_train.columns] = scaler.transform(numeric_train[numeric_train.columns])
    numeric_test[numeric_test.columns] = scaler.transform(numeric_test[numeric_test.columns])
    
    
    return numeric_train,numeric_test

            
def one_hot_encode(categorical_train, categorical_test, column_types, leave_one_out):
    
    columns = categorical_train.columns
    temptrain = pd.DataFrame()
    temptest = pd.DataFrame()
    for x in columns:
        column_info = column_types.loc[column_types.COLUMN_NAME == x,:]
        if column_info["0"].values[0] == "NO_USE":
            temptrain[x] = categorical_train[x]
            temptest[x] = categorical_test[x]
        elif column_info["0"].values[0] == "USE_SPECIAL_FUNCTION":
            code_string = x + "_ONEHOT"
            data_return_train = globals()[code_string](categorical_train[x], leave_one_out)
            data_return_test = globals()[code_string](categorical_test[x], leave_one_out)
            for x in data_return_train.columns:
                temptrain[x] = data_return_train[x]
                temptest[x] = data_return_test[x]
        else:
            categorical_predictor_train = np.array(train[x]).reshape(-1,1)
            categorical_predictor_test = np.array(test[x]).reshape(-1,1)            
            if leave_one_out:
                enc = OneHotEncoder(sparse = False, drop = "first")
            else:
                enc = OneHotEncoder(sparse = False)
            enc.fit(categorical_predictor_train)
            
            categorical_predictor_train = enc.transform(categorical_predictor_train)
            categorical_predictor_test = enc.transform(categorical_predictor_test)
            
            for i in range(0,categorical_predictor_train.shape[1]):
                column_name = "C_" + column_types.loc[column_types.COLUMN_NAME == x, str(i)].values[0]
                temptrain[column_name] = categorical_predictor_train[:,i]
                temptest[column_name] = categorical_predictor_test[:,i]
                
    return temptrain,temptest

def standardize_and_encode (train, test, data_types_filename, categorical_leave_one_out, ordinal_as_numeric):
    
    response_train = train.pop("HALL")
    response_test = test.pop("HALL")
    
    column_types = pd.read_csv(data_types_filename)
    numeric_columns = column_types.loc[column_types.DATA_TYPE != "Categorical", "COLUMN_NAME"].values 
    numeric_columns = np.intersect1d(numeric_columns, train.columns.values)   
    
    numeric_train = train.loc[:, numeric_columns]
    numeric_test = test.loc[:, numeric_columns]
    
    categorical_train = train.drop(numeric_columns, axis = 1)
    categorical_test = test.drop(numeric_columns, axis = 1)   
    
    
    if not ordinal_as_numeric:
        ordinal_columns = column_types.loc[column_types.DATA_TYPE == "Ordinal", "COLUMN_NAME"].values 
        ordinal_columns = np.intersect1d(ordinal_columns, numeric_train.columns.values)   
        
        ordinal_train = numeric_train.loc[:, ordinal_columns]
        ordinal_test = numeric_test.loc[:, ordinal_columns]
        
        numeric_train = numeric_train.drop(ordinal_columns, axis = 1)
        numeric_test = numeric_test.drop(ordinal_columns, axis = 1)
    
    numeric_train,numeric_test = standardize(numeric_train,numeric_test)
    categorical_train,categorical_test = one_hot_encode(categorical_train, categorical_test, column_types, categorical_leave_one_out)
    
    train = pd.concat([numeric_train, categorical_train], axis = 1)
    train = pd.concat([train,response_train], axis = 1)
    
    test = pd.concat([numeric_test, categorical_test], axis = 1)  
    test = pd.concat([test,response_test], axis = 1)
    
    if not ordinal_as_numeric:
        train = pd.concat([train, ordinal_train], axis = 1)
        test = pd.concat([test, ordinal_test], axis = 1)
    
    return train,test

#### In this cell we define our functions for dimensionality reduction

In [171]:
from sklearn.decomposition import PCA

def dimreduc_PCA (train, test, variance_keep):
    non_predictor_variables = ["HALL", "EVENT_ID", "LAST_EVENT_ID", "PATNO"]
    temptrain = train.loc[:, non_predictor_variables]
    temptest = test.loc[:, non_predictor_variables]
    train.drop(non_predictor_variables, axis = 1, inplace = True)
    test.drop(non_predictor_variables, axis = 1, inplace = True)
    
    pca = PCA(svd_solver = "full")
    pca.fit(train)
    
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    if variance_keep == 1:
        index = np.where(cumsum > 0)[0][-1]
    else:    
        index = np.where(cumsum >= variance_keep)[0][0]
    
    train_transform = pca.transform(train)
    test_transform = pca.transform(test)
    
    for i in range(0,index + 1):
        temptrain["PCA" + str(i)] = train_transform[:,i]
        temptest["PCA" + str(i)] = test_transform[:,i]
        
    return temptrain,temptest
    
    

#### In this cell we define our functions for derived longitudinal statistics

In [172]:
from numpy.polynomial import polynomial as np_P

def i_approach (df, numeric_approach):
    
    new_df = pd.DataFrame()
    
    event_id_to_years = {
        "BL" : 0,
        "V04" : 1,
        "V06" : 2,
        "V08" : 3,
        "V10" : 4,
        "V12" : 5,
        "V13" : 6,
        "V14" : 7,
        "V15" : 8
    }
    
    unique_subjects = df.PATNO.unique()  
    for x in unique_subjects:
        #We find i and drop observations after i if applicable
        subject_data = df.loc[df.PATNO == x, :].sort_values("EVENT_ID").reset_index(drop = True)
        #We only drop data if i != LAST_EVENT_ID
        if subject_data.loc[subject_data.HALL == 1, :].index.shape[0] != 0:
            i = subject_data.loc[subject_data.HALL == 1, :].index[0]
            if i != subject_data.index[-1]:
                subject_data.drop(subject_data.index[i +1:], axis = 0, inplace = True)
         
        if subject_data.shape[0] >= 3:
            new_row = {}
            
            years_after_BL = subject_data.apply(lambda row_wise: event_id_to_years [row_wise["EVENT_ID"]], axis = 1)
            years_after_BL = np.array(years_after_BL[0:-1])
            
            
            
            for c in subject_data.columns:
                
                if c == "PATNO":
                    new_row[c] = subject_data.PATNO.values[0]
                elif c == "EVENT_ID" or c == "LAST_EVENT_ID":
                    pass
                elif c == "HALL":
                    new_row[c] = subject_data.HALL.values[-1]
                elif "C_" in c:
                    new_row[c + "_MODE"] = subject_data[c][0:-1].mode()[0]

                else:
                    if numeric_approach["degree"] != -1:
                        X = years_after_BL
                        Y = np.array(subject_data[c][0:-1])   
                        degree = numeric_approach["degree"]
                        coefs = np_P.Polynomial.fit(X,Y,degree).convert().coef

                        for i in range(0,degree +1):
                            if coefs.size <= i:
                                new_row[c + "_DEGREE" + str(i)] = 0
                            else:
                                if i == 0:
                                    new_row[c + "_INTERCEPT"] = coefs[i]
                                else:
                                    new_row[c + "_DEGREE" + str(i)] = coefs[i]

                    if numeric_approach["average"] == 1:
                        new_row[c + "_MEAN"] = subject_data[c][0:-1].mean()

                    if numeric_approach["max"] == 1:
                        new_row[c + "_MAX"] = subject_data[c][0:-1].max()  

                    if numeric_approach["min"] == 1:
                        new_row[c + "_MIN"] = subject_data[c][0:-1].min()  

            new_df = new_df.append(new_row, ignore_index = True)
                    
    return  new_df



def bl_approach (df):
    tempdf = pd.DataFrame(df.PATNO.unique(), columns = ["PATNO"])

    HALL_EVER = []
    for id in df.PATNO.unique():
        if df.loc[(df.PATNO == id) & (df.HALL == 1), "HALL"].empty:
            HALL_EVER.append(0)
        else:
            HALL_EVER.append(1)

    df.drop("HALL", axis = 1, inplace = True)
    tempdf["HALL"] = HALL_EVER
    df = df.merge(tempdf, how = "inner", on = "PATNO") 
    
    #Selecting only Baseline observations
    df = df.loc[df.EVENT_ID == "BL", :]
    #We can then safely drop EVENT_ID and PATNO
    df.drop(["EVENT_ID", "LAST_EVENT_ID"], axis = 1, inplace = True)
    return df

#### In this cell we define a function that prints our a correlation heatmap of our features

In [173]:
def corr_heatmap (df):
    #Using Pearson Correlation
    non_relevant_features = ["PATNO"]
    data = df.drop(non_relevant_features, axis = 1)
    plt.figure(figsize=(12,10))
    cor = data.corr()
    sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
    plt.show()

#### In this cell we define our train test split function

In [174]:
'''
We split into training and test so that we base standardization or dimensionality reduction mapping on training set only
We have to ensure that all longitudinal observations from one patient ends up in the same sample, we therefore split on PATNOs
We stratify our sampling by converting our response to HALL_EVER. 
This ensures an equal distribution of subjects who never experience hallucinations and those who do in both training and test
Both longitudinal and non-longitudinal models predict on HALL_EVER, although in slightly different forms 
'''
def split_data (data):
    tempdata = pd.DataFrame(data.PATNO.unique(), columns = ["PATNO"])

    HALL_EVER = []
    for id in data.PATNO.unique():

        if data.loc[(data.PATNO == id) & (data.HALL == 1), "HALL"].empty:
            HALL_EVER.append(0)
        else:
            HALL_EVER.append(1)

    Y = HALL_EVER
    X = tempdata

    train, test, _, _ = train_test_split( X, Y, test_size= TEST_PORTION, random_state= 1, stratify= Y)

    train = data.merge(train, how = "inner", on = "PATNO")
    test = data.merge(test, how = "inner", on = "PATNO")
    
    return train,test

#### In this cell we do the final preprocessing as dictated by the flags set

In [175]:
#We split our data into training and test sets
train,test = split_data(data)

#Derived Scores option:
if DERIVED_TOTAL_SCORES != 0:
    if DERIVED_TOTAL_SCORES == 1: 
        derived_values(train, False)
        derived_values(test, False) 
    elif DERIVED_TOTAL_SCORES == 2:
        derived_values(train, True)
        derived_values(test, True) 
    train,test = standardize_and_encode(train,test,"Column_Data_Types_Final.csv", ONE_HOT_LEAVE_ONE_OUT, STANDARDIZE_ORDINAL)  
else:
    
    train,test = standardize_and_encode(train,test,"Column_Data_Types.csv", ONE_HOT_LEAVE_ONE_OUT, STANDARDIZE_ORDINAL)   
    
#Dimensionality reduction option:
if DIMENSIONALITY_REDUCTION != 0:
    if DIMENSIONALITY_REDUCTION == 1:
        train,test = dimreduc_PCA(train,test, VARIANCE_KEEP)
        

#Longitudinal Statistics
if LONGITUDINAL_STATISTICS != 0:
    if LONGITUDINAL_STATISTICS == 1:
        train = i_approach(train,NUMERIC_APPROACH)
        test = i_approach(test,NUMERIC_APPROACH)
else: 
    train = bl_appraoch(train)
    test = bl_approach(test)

    
if SHOW_HEATMAP:
    corr_heatmap(train)
    
    
train.to_csv(output_filepath + '/train.csv', index = False)

In [176]:
train.head()

,HALL,PATNO,PCA0_DEGREE1,PCA0_INTERCEPT,PCA0_MAX,PCA0_MEAN,PCA0_MIN,PCA10_DEGREE1,PCA10_INTERCEPT,PCA10_MAX,PCA10_MEAN,PCA10_MIN,PCA11_DEGREE1,PCA11_INTERCEPT,PCA11_MAX,PCA11_MEAN,PCA11_MIN,PCA12_DEGREE1,PCA12_INTERCEPT,PCA12_MAX,PCA12_MEAN,PCA12_MIN,PCA13_DEGREE1,PCA13_INTERCEPT,PCA13_MAX,PCA13_MEAN,PCA13_MIN,PCA14_DEGREE1,PCA14_INTERCEPT,PCA14_MAX,PCA14_MEAN,PCA14_MIN,PCA15_DEGREE1,PCA15_INTERCEPT,PCA15_MAX,PCA15_MEAN,PCA15_MIN,PCA16_DEGREE1,PCA16_INTERCEPT,PCA16_MAX,PCA16_MEAN,PCA16_MIN,PCA17_DEGREE1,PCA17_INTERCEPT,PCA17_MAX,PCA17_MEAN,PCA17_MIN,PCA18_DEGREE1,PCA18_INTERCEPT,PCA18_MAX,PCA18_MEAN,PCA18_MIN,PCA19_DEGREE1,PCA19_INTERCEPT,PCA19_MAX,PCA19_MEAN,PCA19_MIN,PCA1_DEGREE1,PCA1_INTERCEPT,PCA1_MAX,PCA1_MEAN,PCA1_MIN,PCA20_DEGREE1,PCA20_INTERCEPT,PCA20_MAX,PCA20_MEAN,PCA20_MIN,PCA21_DEGREE1,PCA21_INTERCEPT,PCA21_MAX,PCA21_MEAN,PCA21_MIN,PCA22_DEGREE1,PCA22_INTERCEPT,PCA22_MAX,PCA22_MEAN,PCA22_MIN,PCA23_DEGREE1,PCA23_INTERCEPT,PCA23_MAX,PCA23_MEAN,PCA23_MIN,PCA24_DEGREE1,PCA24_INTERCEPT,PCA24_MAX,PCA24_MEAN,PCA24_MIN,PCA25_DEGREE1,PCA25_INTERCEPT,PCA25_MAX,PCA25_MEAN,PCA25_MIN,PCA26_DEGREE1,PCA26_INTERCEPT,PCA26_MAX,PCA26_MEAN,PCA26_MIN,PCA27_DEGREE1,PCA27_INTERCEPT,PCA27_MAX,PCA27_MEAN,PCA27_MIN,PCA28_DEGREE1,PCA28_INTERCEPT,PCA28_MAX,PCA28_MEAN,PCA28_MIN,PCA29_DEGREE1,PCA29_INTERCEPT,PCA29_MAX,PCA29_MEAN,PCA29_MIN,PCA2_DEGREE1,PCA2_INTERCEPT,PCA2_MAX,PCA2_MEAN,PCA2_MIN,PCA30_DEGREE1,PCA30_INTERCEPT,PCA30_MAX,PCA30_MEAN,PCA30_MIN,PCA31_DEGREE1,PCA31_INTERCEPT,PCA31_MAX,PCA31_MEAN,PCA31_MIN,PCA32_DEGREE1,PCA32_INTERCEPT,PCA32_MAX,PCA32_MEAN,PCA32_MIN,PCA33_DEGREE1,PCA33_INTERCEPT,PCA33_MAX,PCA33_MEAN,PCA33_MIN,PCA3_DEGREE1,PCA3_INTERCEPT,PCA3_MAX,PCA3_MEAN,PCA3_MIN,PCA4_DEGREE1,PCA4_INTERCEPT,PCA4_MAX,PCA4_MEAN,PCA4_MIN,PCA5_DEGREE1,PCA5_INTERCEPT,PCA5_MAX,PCA5_MEAN,PCA5_MIN,PCA6_DEGREE1,PCA6_INTERCEPT,PCA6_MAX,PCA6_MEAN,PCA6_MIN,PCA7_DEGREE1,PCA7_INTERCEPT,PCA7_MAX,PCA7_MEAN,PCA7_MIN,PCA8_DEGREE1,PCA8_INTERCEPT,PCA8_MAX,PCA8_MEAN,PCA8_MIN,PCA9_DEGREE1,PCA9_INTERCEPT,PCA9_MAX,PCA9_MEAN,PCA9_MIN
0,1.0,3400.0,0.240850,-2.267868,-0.124627,-1.545317,-2.505983,0.096395,0.426841,1.560093,0.716027,0.042416,0.013313,-0.320167,0.276721,-0.280227,-0.912798,0.307523,-0.339543,2.201500,0.583025,-0.750834,0.095399,0.798749,1.785773,1.084946,0.494741,-0.154717,-0.145352,-0.136805,-0.609503,-1.425387,0.143898,-0.141510,1.019473,0.290183,-0.619457,0.029196,-0.151786,0.204004,-0.064196,-0.384961,0.119997,-0.189019,0.714989,0.170972,-0.378129,-0.088969,-0.050991,0.307128,-0.317897,-1.354148,0.005582,-0.685386,-0.005674,-0.668639,-1.515636,-0.370375,0.009445,0.857251,-1.101680,-2.104754,0.049190,0.189915,0.786056,0.337484,-0.176249,0.024172,0.113907,0.908660,0.186422,-0.776374,-0.028360,-0.022633,0.625685,-0.107712,-0.781834,0.086542,-0.384868,0.270691,-0.125243,-0.621251,-0.086043,0.296905,0.381174,0.038776,-0.348859,-0.019685,-0.019139,1.075462,-0.078194,-0.377581,0.052229,-0.022260,0.311970,0.134426,-0.218320,-0.089060,0.191733,0.267066,-0.075447,-0.507327,-3.735479e-17,1.201367e-16,2.962725e-16,8.072314e-18,-1.478167e-16,-5.383114e-17,3.432574e-16,3.425577e-16,1.817640e-16,-9.934009e-17,0.080878,0.984586,2.014683,1.227219,0.787613,-1.193357e-16,1.803806e-16,1.704364e-16,-1.776265e-16,-6.453279e-16,-5.746505e-18,4.841363e-17,3.022659e-16,3.117412e-17,-3.638679e-16,3.142861e-17,-2.148672e-16,-2.480355e-17,-1.205814e-16,-3.023593e-16,-1.160025e-16,3.254074e-16,2.844864e-16,-2.259999e-17,-4.156156e-16,-0.035634,-0.704856,0.288284,-0.811758,-1.784811,0.085030,0.183539,1.247895,0.438628,-0.395599,-0.371251,2.599950,2.465223,1.486196,-0.003185,-0.030914,-0.832698,-0.413417,-0.925439,-1.290576,-0.074362,-0.649066,-0.123811,-0.872153,-1.488034,0.254668,-0.483677,1.306339,0.280327,-0.890895,-0.035778,-0.248688,0.045471,-0.356021,-0.705105
1,1.0,3403.0,0.411128,1.273965,5.082815,2.712912,0.687486,0.014710,0.318969,0.940406,0.370453,-0.311682,0.006614,-0.411509,0.093375,-0.388360,-0.871439,-0.116483,-0.182311,0.719801,-0.590001,-1.255555,-0.079762